In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv
/kaggle/input/d/hrishitkaggle/titanic/train.csv
/kaggle/input/d/hrishitkaggle/titanic/test.csv
/kaggle/input/d/hrishitkaggle/titanic/gender_submission.csv


In [2]:
from sklearn.impute import SimpleImputer

df_1 = pd.read_csv("/kaggle/input/titanic/train.csv")
df_2 = pd.read_csv("/kaggle/input/titanic/test.csv")
df_1["Sex"] = df_1["Sex"].replace({"male":0, "female":1})
train_cat = df_1[['Embarked','Parch','SibSp','Pclass']]
train_cat = train_cat.dropna(axis=0)
train_cont = df_1[['Age','Fare']]
mean_imputer = SimpleImputer(strategy='mean')
train_cont['Age'] = mean_imputer.fit_transform(train_cont['Age'].values.reshape(-1,1))

df_2["Sex"] = df_2["Sex"].replace({"male":0, "female":1})
test_cat = df_2[['Embarked','Parch','SibSp','Pclass']]
test_cat =test_cat.dropna(axis=0)
test_cont = df_2[['Age','Fare']]
test_cont['Age'] = mean_imputer.fit_transform(test_cont['Age'].values.reshape(-1,1))
test_cont['Fare'] = mean_imputer.fit_transform(test_cont['Fare'].values.reshape(-1,1))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the doc

In [3]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder

scaler = StandardScaler()
scaled_cont_vars_1 = scaler.fit_transform(train_cont)
scaled_cont_vars_1 = pd.DataFrame(scaled_cont_vars_1)
scaled_cont_vars_2 = scaler.fit_transform(test_cont)
scaled_cont_vars_2 = pd.DataFrame(scaled_cont_vars_2)

encoder = OneHotEncoder()
one_hot_cat_vars_1 = encoder.fit_transform(train_cat)
one_hot_cat_vars_1 = pd.DataFrame(one_hot_cat_vars_1.toarray())
one_hot_cat_vars_2 = encoder.fit_transform(test_cat)
one_hot_cat_vars_2 = pd.DataFrame(one_hot_cat_vars_2.toarray())

x_train = pd.concat([df_1["Sex"],scaled_cont_vars_1, one_hot_cat_vars_1], axis=1)
y_train = df_1.iloc[:, [1]]
x_train = x_train.drop(index=[889,890])
y_train = y_train.drop(index=[889,890])
x_test = pd.concat([df_2["Sex"],scaled_cont_vars_2, one_hot_cat_vars_2], axis=1)
a1, a2 = x_train.align(x_test, join="left", axis=1)

     Sex         0         1    0    1    2    3    4    5    6  ...   10  \
0      0 -0.592481 -0.502445  0.0  0.0  1.0  1.0  0.0  0.0  0.0  ...  0.0   
1      1  0.638789  0.786845  1.0  0.0  0.0  1.0  0.0  0.0  0.0  ...  0.0   
2      1 -0.284663 -0.488854  0.0  0.0  1.0  1.0  0.0  0.0  0.0  ...  1.0   
3      1  0.407926  0.420730  0.0  0.0  1.0  1.0  0.0  0.0  0.0  ...  0.0   
4      0  0.407926 -0.486337  0.0  0.0  1.0  1.0  0.0  0.0  0.0  ...  1.0   
..   ...       ...       ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
884    0 -0.361618 -0.506472  0.0  0.0  1.0  1.0  0.0  0.0  0.0  ...  1.0   
885    1  0.715743 -0.061999  0.0  0.0  1.0  1.0  0.0  0.0  0.0  ...  1.0   
886    0 -0.207709 -0.386671  0.0  0.0  1.0  0.0  0.0  1.0  0.0  ...  0.0   
887    1 -0.823344 -0.044381  1.0  0.0  0.0  1.0  0.0  0.0  0.0  ...  1.0   
888    1  0.000000 -0.176263  0.0  1.0  0.0  1.0  0.0  0.0  0.0  ...  1.0   

      11   12   13   14   15   16   17   18   19  
0    1.0  0.0  0.0  0.0 

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(a1, y_train)

new_col = ['PassengerId', 'Survived']
df_2['Survived'] = model.predict(a2)
y_test = df_2['Survived']
df_2[new_col].to_csv('submission.csv', index=False)

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
